## Initial Setup

1. Download the Murofet version 0.0.0.8 reference sample from [MalwareBazaar](https://bazaar.abuse.ch/sample/1a30a352a65cf6ad2b9b8266617709672c777e58edaa625946d77aca427cd352/).
2. Unpack the sample with [UNPACME](https://www.unpac.me/results/ffa339ca-dd22-43a0-a2b8-13e8d93d2db1#/).

In [1]:
# Install required Python modules
import sys
!{sys.executable} -m pip install pefile

# Import required Python modules
import datetime
import hashlib
import re
import struct

import pefile

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Load the unpacked sample
fp = open("a763710ca7fa7d1f48c9713c9aa5ee037f0038a7140b61b1381ba5d5fa7a0ab9.bin", "rb")
sample = fp.read()
fp.close()

pe = pefile.PE(data=sample)

## Decrypt Strings

Pseudocode:

    int __usercall CryptedStrings::_getA@<eax>(unsigned __int16 id@<ax>, int buffer@<edi>)
    {
      STRINGINFO *s; // eax
      int i; // ecx
      int i2; // esi
      char pb; // dl
      int size; // eax

      s = &dword_4045D0[2 * id];
      for ( i = 0; i < *&s->size; *(i2 + buffer) = pb )
      {
        i2 = i;
        pb = i ^ s->key ^ *(s->encodedString + i);
        ++i;
      }
      size = *&s->size;
      *(size + buffer) = 0;
      return size;
    }

Disassembly:

    .text:00405871                               CryptedStrings___getA proc near
    .text:00405871
    .text:00405871 0F B7 C0                                      movzx   eax, ax
    .text:00405874 8D 04 C5 D0 45 40 00                          lea     eax, dword_4045D0[eax*8]
    .text:0040587B 33 D2                                         xor     edx, edx
    .text:0040587D 33 C9                                         xor     ecx, ecx
    .text:0040587F 66 3B 50 02                                   cmp     dx, [eax+2]
    .text:00405883 73 19                                         jnb     short loc_40589E
    .text:00405885 56                                            push    esi
    .text:00405886
    .text:00405886                               loc_405886:
    .text:00405886 8B 50 04                                      mov     edx, [eax+4]
    .text:00405889 0F B7 F1                                      movzx   esi, cx
    .text:0040588C 8A 14 32                                      mov     dl, [edx+esi]
    .text:0040588F 32 10                                         xor     dl, [eax]
    .text:00405891 32 D1                                         xor     dl, cl
    .text:00405893 41                                            inc     ecx
    .text:00405894 88 14 3E                                      mov     [esi+edi], dl
    .text:00405897 66 3B 48 02                                   cmp     cx, [eax+2]
    .text:0040589B 72 E9                                         jb      short loc_405886
    .text:0040589D 5E                                            pop     esi
    .text:0040589E
    .text:0040589E                               loc_40589E:
    .text:0040589E 0F B7 40 02                                   movzx   eax, word ptr [eax+2]
    .text:004058A2 C6 04 38 00                                   mov     byte ptr [eax+edi], 0
    .text:004058A6 C3                                            retn
    .text:004058A6                               CryptedStrings___getA endp

In [3]:
def decrypt_string(pe, id_):
    match = re.search(rb'\x8d\x04\xc5(?P<enc_strings_addr>[\s\S]{4})', 
                      pe.get_memory_mapped_image())
    enc_strings_addr = struct.unpack("I", match.group("enc_strings_addr"))[0]
    
    enc_string_addr = enc_strings_addr + 8 * id_
    enc_string_rva = enc_string_addr - pe.OPTIONAL_HEADER.ImageBase
    
    xor_key = pe.get_word_at_rva(enc_string_rva)
    encbuf_len = pe.get_word_at_rva(enc_string_rva + 2)
    encbuf_addr = pe.get_dword_at_rva(enc_string_rva + 4)
    encbuf_rva = encbuf_addr - pe.OPTIONAL_HEADER.ImageBase
    encbuf = pe.get_data(encbuf_rva, encbuf_len)
    
    plainbuf = b""
    for i in range(len(encbuf)):
        pb = (i ^ xor_key ^ encbuf[i]) & 0xff
        plainbuf += struct.pack("B", pb)
        
    return plainbuf.decode()


for i in range(264):
    plain_string = decrypt_string(pe, i)
    print(f"{i}: {plain_string}")

0: https://securentrycorp.*/Authentication/zbf/index?*domainId=*
1: nmRef
2: https://*/ebc_ebc1961/ebc1961.asp*=RemoteLogon*
3: nmUID
4: nmRef
5: %s=%s; path=/;
6: CCDUMP: %S
Trash: %S
7: %08x.%08x
8: *.%08x
9: socks
10: vnc
11: Global\%08X%08X%08X
12: http://%s/news/
13: %s?s=%u
14: If-None-Match: %s

15: %s://%s:%s@%s/
16: ftp
17: pop3
18: anonymous
19: %s%s
Referer: %S
User input: %s
%sPOST data:

%S
20: *EMPTY*
21: *UNKNOWN*
22:  *BLOCKED*
23: Content-Type: %s

24: Accept-Encoding: gzip, deflate

25: X-ID: %s=%s

26: X-BC
27: application/x-www-form-urlencoded
28: HTTP authentication: username="%s", password="%s"

29: HTTP authentication (encoded): %S

30: Accept-Encoding: identity

31: Accept-Encoding
32: identity
33: TE:

34: TE
35: If-Modified-Since:

36: If-Modified-Since
37: Transfer-Encoding
38: chunked
39: Content-Length
40: Content-Type
41: Host
42: Referer
43: Authorization
44: HTTP/1.1
45: https://
46: http://
47: PR_GetNameForIdentity
48: PR_SetError
49: PR_GetError
50: P

## Extract Version

Pseudocode:

    ...
    version[0] = 8;
    r = BinStorage::_addItem(binStorage, 4u, 10003, 0x20000, version);
    ...

Disassembly:

    ...
    .text:00413695 8D 4D F8                                      lea     ecx, [ebp+version]
    .text:00413698 51                                            push    ecx
    .text:00413699 56                                            push    esi
    .text:0041369A 68 13 27 00 00                                push    2713h
    .text:0041369F 6A 04                                         push    4
    .text:004136A1 5B                                            pop     ebx
    .text:004136A2 C7 45 F8 08 00 00 00                          mov     [ebp+version], 8
    .text:004136A9 E8 88 31 FF FF                                call    BinStorage___addItem
    ...

In [4]:
match = re.search(rb'\x68\x13\x27\x00\x00[\s\S]{1,8}?\xc7\x45[\s\S](?P<version>[\s\S]{4})', 
                  pe.get_memory_mapped_image())
version_bytes = match.group("version")
version = ".".join([str(b) for b in version_bytes[::-1]])
print(version)

0.0.0.8


## Decrypt BaseConfig

Pseudocode:

    int __usercall Core::getBaseConfig@<eax>(int bc@<eax>)
    {
      __int64 v1; // rax
      int v2; // esi
      int v3; // ecx

      v1 = Mem::_copy(bc, dword_402EB0, 700);
      v2 = HIDWORD(v1);
      v3 = dword_42A9B0 + dword_42A504 - v1;
      do
      {
        *v1 ^= *(v3 + v1);
        LODWORD(v1) = v1 + 1;
        --v2;
      }
      while ( v2 );
      return v1;
    }

Disassembly:

    .text:0040B190                               Core__getBaseConfig proc near
    .text:0040B190
    .text:0040B190 56                                            push    esi
    .text:0040B191 BA BC 02 00 00                                mov     edx, 2BCh
    .text:0040B196 52                                            push    edx
    .text:0040B197 68 B0 2E 40 00                                push    offset dword_402EB0
    .text:0040B19C 50                                            push    eax
    .text:0040B19D E8 99 37 01 00                                call    Mem___copy
    .text:0040B1A2 8B 0D 04 A5 42 00                             mov     ecx, dword_42A504
    .text:0040B1A8 03 0D B0 A9 42 00                             add     ecx, dword_42A9B0
    .text:0040B1AE 8B F2                                         mov     esi, edx
    .text:0040B1B0 2B C8                                         sub     ecx, eax
    .text:0040B1B2
    .text:0040B1B2                               loc_40B1B2:
    .text:0040B1B2 8A 14 01                                      mov     dl, [ecx+eax]
    .text:0040B1B5 30 10                                         xor     [eax], dl
    .text:0040B1B7 40                                            inc     eax
    .text:0040B1B8 4E                                            dec     esi
    .text:0040B1B9 75 F7                                         jnz     short loc_40B1B2
    .text:0040B1BB 5E                                            pop     esi
    .text:0040B1BC C3                                            retn
    .text:0040B1BC                               Core__getBaseConfig endp

In [5]:
match = re.search(rb'\xba(?P<encbuf_len>[\s\S]{2}\x00\x00)\x52\x68(?P<encbuf_addr>[\s\S]{4})',
                   pe.get_memory_mapped_image())

encbuf_len = struct.unpack("I", match.group("encbuf_len"))[0]
encbuf_addr = struct.unpack("I", match.group("encbuf_addr"))[0]
encbuf_rva = encbuf_addr - pe.OPTIONAL_HEADER.ImageBase
encbuf = pe.get_data(encbuf_rva, encbuf_len)

# #define PESECTION_OF_BASECONFIG_KEY 2
section = pe.sections[2]
xor_key = pe.get_data(section.VirtualAddress, encbuf_len)

base_config = b""
for i, eb in enumerate(encbuf):
    pb = eb ^ xor_key[i % len(xor_key)]
    base_config += struct.pack("B", pb)
    
print(base_config)

b'\xac\x9e\xe4s\x93\x00\xc0\x195\x9b\xa9kT<\x0b\xb7W\xbe!\xdd\x05\x00x\x00\xf5\xe48\x89\x9b\x84\xdd?\xc3kJ\xd6\xaa\xf2@\x95\xca\xbb\xaaM\xffC\xdd\x8f\xe9\xba\x1e\x1c\x91\x12d\x9dOV`\x95\xeb=\xbb\x93\x02\xff\x9f\xac\xabgx(2\x009\x00.\x000\x007\x00F\x00E\x00D\x00\x00\x00^`;6\x17\xac2\xff\xeb\xdc\xde\x86\xf1;\xae\x90\xa7\xdf\xb1\xff7\xdb\xfb\xa5\xf8\xb6IZ8\x1c\xe8L9\xd4\xd2\x97y\xf2\x1b\x166N\x8bx24\x8d\x96\xb4]G\x03\xcf\x1e\x0b\xe5\xe6\xe0\xee\x04p\xd0\xb6!\xf9\xd8\xfa\x89i\xcd\x84\xeaopwhmilxsjkqdqe.info\x00\x07<\x11\xa7S\xf94p\xa56#\xeb\x98ss-|\t \x11OX#\x7f\xe9!B\xedv1,%)@\xc5I/\x18\xbf\x15\xe2\xfaje@=\xcd\xf7\x9b8\xf5q\x91b\x8d\xa6?\xb8\xaf\x15\xb2_\xfa\xb2\x9e\x8b\xe5\xeaS\x9fR\xe1\xec\x96g\xfe\xca\xa1\x99\xa9A\x11\tS\x90x\x14\xb7\x17\x0b%`\xc4K\xcel\xcf\x02:\x15es> w~X,\xe5f\\\xc9B;\xb5|\xc8\xff\x18\x00\x97\xe6\xb1\xb7bm\x94\xfd\x84g!d0\xbcZ\xce+\x8ex\x01\xdd\x11\xaaD\xd1&\xb9]\xee\xe7\xb2l\xe1\x1eO\xae\xd6\xa8\x1b\x9f\xd0\xc6k\xad1v)\xa08\n\xa5\xd7\x06\x99_<\xe2G\x9a*\x1c\x80(\xf4

## Parse BaseConfig

In [6]:
flags = struct.unpack("I", base_config[0x298:0x298+4])[0]
print(f"flags: 0x{flags:x}\n")

delayStats = struct.unpack("I", base_config[0x258:0x258+4])[0]
print(f"delayStats: {delayStats}\n")

defaultConfig = base_config[0xa2:0xa2+101].split(b"\x00")[0].decode().replace("http", "hxxp")
print(f"defaultConfig: {defaultConfig}\n")

baseKey = base_config[0x118:0x118+258]
print(f"baseKey: {baseKey}\n")

defaultBotnet = base_config[0x48:0x48+40].split(b"\x00\x00")[0].decode()
print(f"defaultBotnet: {defaultBotnet}\n")

delayReport = struct.unpack("I", base_config[0x2a4:0x2a4+4])[0]
print(f"delayReport: {delayReport}\n")

delayConfig = struct.unpack("I", base_config[0x14:0x14+4])[0]
print(f"delayConfig: {delayConfig}\n")

unknown_word_at_offset_0x70 = struct.unpack("H", base_config[0x70:0x70+2])[0]
print(f"unknown_word_at_offset_0x70: {unknown_word_at_offset_0x70}\n")

offset_0xb4_maybe_corefile = base_config[0xb4:].split(b"\x00")[0].decode()
print(f"offset_0xb4_maybe_corefile: {offset_0xb4_maybe_corefile}")

flags: 0x0

delayStats: 1966081

defaultConfig: opwhmilxsjkqdqe.info

baseKey: b'\x02:\x15es> w~X,\xe5f\\\xc9B;\xb5|\xc8\xff\x18\x00\x97\xe6\xb1\xb7bm\x94\xfd\x84g!d0\xbcZ\xce+\x8ex\x01\xdd\x11\xaaD\xd1&\xb9]\xee\xe7\xb2l\xe1\x1eO\xae\xd6\xa8\x1b\x9f\xd0\xc6k\xad1v)\xa08\n\xa5\xd7\x06\x99_<\xe2G\x9a*\x1c\x80(\xf4\x0cACP\'\x89\xacW\x19\xa6\xb3\x81\xca\x8d\xd2\xe8\xd5\x1f\xe3\x833\xa9-\x8fYr\x8cc\xd3\xcb\x82\x10\x95\x1d\xda#\xea\x9d\xf6$\x9e\xab\xd9}V\xf1\xe4\xb0\xb8\xba\x07\xa3u\x059n\x88a\xf2\xfe\xf9\xe9\x08KT\xa7\x86\xd8\xbe5N\xbbjQ\xed2\xf7\xf8\t\x98@\x90^?o\xa4\x1a\x04{\xf5\xc4\xfa\xfb\x17q\xfc\x87\xbdp\xc3J\r\xbf\xeb7\xcd\xa2\xc5i\x8a\xdb\x0b\x7f\x12y\xc7\xc2`t\xf0I\x93R\x9b\x96"6\x03\x14E\xa1\x13\x9c\x92\xe0\x0e.zU\x0f\xcf\xc0%M\xaf[\x16F\x8bh\xf3/\xdc\xec=S\xb4\x85\xc1\xdf\xd4\xefH\x91\xb6L\xde\xcc4\x00\x00'

defaultBotnet: 2 9 . 0 7 F E D

delayReport: 1310725

delayConfig: 7864325

unknown_word_at_offset_0x70: 42491

offset_0xb4_maybe_corefile: fo


## Domain Generation Algorithm

Pseudocode:

    unsigned __int8 __userpurge domain_generation_algorithm@<al>(SYSTEMTIME *date@<eax>, int domain, unsigned int index)
    {
      char wMonth; // cl
      s16 *p_s16; // eax
      int v5; // ecx
      int digest_len_cp; // edi
      unsigned __int8 result; // al
      unsigned __int8 domain_len; // bl
      BYTE *digest_cp; // edx
      unsigned __int8 tmp; // al
      unsigned __int8 domain_len_p1; // bl
      _DWORD *p_tld; // ecx
      unsigned __int8 final_domain_len; // bl
      BYTE digest[16]; // [esp+10h] [ebp-28h] BYREF
      s16 s16; // [esp+20h] [ebp-18h] BYREF
      HCRYPTHASH phHash; // [esp+30h] [ebp-8h] BYREF
      char digest_ok; // [esp+37h] [ebp-1h]

      s16.field_0 = LOBYTE(date->wYear) + 0x30;
      wMonth = date->wMonth;
      s16.field_2 = date->wDay;
      s16.field_1 = wMonth;
      s16.field_4 = index & 0xFFFFFFFE;
      s16.field_3 = 0;
      p_s16 = &s16;
      v5 = 2;
      do
      {
        *&p_s16->field_0 ^= 0xED79A19C;
        p_s16 = (p_s16 + 4);
        --v5;
      }
      while ( v5 );
      digest_ok = 0;
      if ( !CryptAcquireContextW(&s16.phProv, 0, 0, 1u, 0xF0000040) )
        return 0;
      digest_len_cp = 16;
      if ( CryptCreateHash(*&s16.phProv, CALG_MD5, 0, 0, &phHash) )
      {
        *&s16.digest_len = 16;
        if ( CryptHashData(phHash, &s16, 8u, 0)
          && CryptGetHashParam(phHash, HP_HASHVAL, digest, &s16.digest_len, 0)
          && *&s16.digest_len == 16 )
        {
          digest_ok = 1;
        }
        CryptDestroyHash(phHash);
      }
      CryptReleaseContext(*&s16.phProv, 0);
      if ( !digest_ok )
        return 0;
      domain_len = 0;
      digest_cp = digest;
      do
      {
        tmp = (*digest_cp >> 4) + (*digest_cp & 0xF) + 'a';
        if ( tmp <= 'z' )
          *(domain_len++ + domain) = tmp;
        ++digest_cp;
        --digest_len_cp;
      }
      while ( digest_len_cp );
      *(domain_len + domain) = '.';
      domain_len_p1 = domain_len + 1;
      p_tld = (domain + domain_len_p1);
      final_domain_len = domain_len_p1 + 3;
      if ( index % 5 )
      {
        if ( (index & 3) != 0 )
        {
          if ( index % 3 )
            *p_tld = (index & 1) != 0 ? 'moc' : 'ten';
          else
            *p_tld = 'gro';
        }
        else
        {
          *p_tld = 'ofni';
          ++final_domain_len;
        }
      }
      else
      {
        *p_tld = 'zib';
      }
      result = final_domain_len;
      *(final_domain_len + domain) = 0;
      return result;
    }

Disassembly:

    ...
    .text:0041DC5D
    .text:0041DC5D                               loc_41DC5D:
    .text:0041DC5D 81 30 9C A1 79 ED             xor     dword ptr [eax], 0ED79A19Ch
    .text:0041DC63 83 C0 04                      add     eax, 4
    .text:0041DC66 49                            dec     ecx
    .text:0041DC67 75 F4                         jnz     short loc_41DC5D
    ...

In [7]:
def dga(xor_key, date, index):
    s8 = b"" 
    s8 += struct.pack("B", ((date.year & 0xff) + 0x30) & 0xff)
    s8 += struct.pack("B", date.month)
    s8 += struct.pack("B", date.day)
    s8 += struct.pack("B", 0)
    s8 += struct.pack("I", (index & 0xfffffffe))

    s8_encrypted = b"" 
    for i in range(0, 8, 4): 
        pd = struct.unpack("I", s8[i:i+4])[0]
        ed = pd ^ xor_key
        s8_encrypted += struct.pack("I", ed) 

    md5 = hashlib.md5()
    md5.update(s8_encrypted)
    digest = md5.digest()

    domain = ""
    for b in digest:
        tmp = (b >> 4) + (b & 0xf) + ord("a")
        if tmp <= ord("z"):
            domain += chr(tmp)

    if index % 5:
        if index & 3 != 0:
            if index % 3:
                if index & 1 != 0:
                    tld = ".com"
                else:
                    tld = ".net"
            else:
                tld = ".org"
        else:
            tld = ".info"
    else:
        tld = ".biz"

    domain += tld 

    return domain


match = re.search(rb'\x81\x30(?P<xor_key>[\s\S]{4})\x83\xc0\x04', pe.get_memory_mapped_image())
xor_key = struct.unpack("I", match.group("xor_key"))[0]
print(f"xor_key: 0x{xor_key:x}")

date = datetime.datetime(2022, 12, 3)
print(f'date: {date.strftime("%Y-%m-%d")}')

for index in range(5):
    print(dga(xor_key, date, index))

xor_key: 0xed79a19c
date: 2022-12-03
iqrqzvqpsrrqqvpq.biz
iqrqzvqpsrrqqvpq.com
iusvhttmymmmgre.net
iusvhttmymmmgre.org
olpdtqklptomphql.info


## Download DynamicConfig

I haven't been able to find a DynamicConfig file for any sample of Murofet. If you have any of these files in your archive, please reach out to curator@zeusmuseum.com / [@tildedennis](https://twitter.com/tildedennis).

## Decrypt DynamicConfig

Pseudocode:

    ...
    Mem::_copy(v7, rc4Key, 0x102u);
    Crypt::_rc4(v3, binStorage, dataSize);
    if ( dataSize ) // inlined Crypt::_visualDecrypt
    {
      v4 = dataSize - 1;
      if ( dataSize != 1 )
      {
        do
        {
          *(v4 + binStorage) ^= *(v4 + binStorage - 1);
          --v4;
        }
        while ( v4 );
      }
    }
    ...

## Commands

Pseudocode:

    ...
    ci = 0;
    while ( 1 )
    {
        CryptedStrings::_getW(word_402DE8[4 * ci], commandNameBuffer);
        if ( !lstrcmpiW(*args, commandNameBuffer) )
          break;
        if ( ++ci >= 25 )
          goto LABEL_35;
    }
    ...

Disassembly:

    ...
    .text:0041D653 33 FF                                         xor     edi, edi
    .text:0041D655
    .text:0041D655                               loc_41D655:
    .text:0041D655 66 8B 04 FD E8 2D 40 00                       mov     ax, ds:word_402DE8[edi*8]
    .text:0041D65D 8D B5 54 FD FF FF                             lea     esi, [ebp+commandNameBuffer]
    .text:0041D663 E8 3F 82 FE FF                                call    CryptedStrings___getW
    .text:0041D668 8B C6                                         mov     eax, esi
    .text:0041D66A 50                                            push    eax
    .text:0041D66B FF 33                                         push    dword ptr [ebx]
    .text:0041D66D FF 15 30 12 40 00                             call    ds:lstrcmpiW
    .text:0041D673 85 C0                                         test    eax, eax
    .text:0041D675 74 08                                         jz      short loc_41D67F
    .text:0041D677 47                                            inc     edi
    .text:0041D678 83 FF 19                                      cmp     edi, 19h
    .text:0041D67B 72 D8                                         jb      short loc_41D655
    .text:0041D67D EB 1F                                         jmp     short loc_41D69E
    ...

In [8]:
match = re.search(rb'\x66\x8b\x04\xfd(?P<indexes_addr>[\s\S]{4})', pe.get_memory_mapped_image())
indexes_addr = struct.unpack("I", match.group("indexes_addr"))[0]

for i in range(25):
    index_addr = indexes_addr + i * 8
    index_rva = index_addr - pe.OPTIONAL_HEADER.ImageBase
    enc_string_index = pe.get_word_at_rva(index_rva)
    
    plain_string = decrypt_string(pe, enc_string_index)
    print(f"{enc_string_index}: {plain_string}")

201: os_shutdown
202: os_reboot
203: bot_uninstall
204: bot_update
205: bot_bc_add
206: bot_bc_remove
207: bot_httpinject_disable
208: bot_httpinject_enable
209: fs_path_get
210: fs_search_add
211: fs_search_remove
212: user_destroy
213: user_logoff
214: user_execute
215: user_cookies_get
216: user_cookies_remove
217: user_certs_get
218: user_certs_remove
219: user_url_block
220: user_url_unblock
221: user_homepage_set
222: user_ftpclients_get
223: user_emailclients_get
224: user_flashplayer_get
225: user_flashplayer_remove
